# Capstone Journey Notebook

In [ ]:
import pandas as pd, numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.metrics import roc_auc_score, brier_score_loss, precision_recall_fscore_support, confusion_matrix
from xgboost import XGBClassifier
import shap, matplotlib.pyplot as plt, joblib, warnings
warnings.filterwarnings("ignore")
np.random.seed(42)

DATA_PATH = Path("data/diabetes_prediction_dataset.csv")
MODELS_DIR = Path("models"); MODELS_DIR.mkdir(exist_ok=True)
RESULTS_DIR = Path("results"); RESULTS_DIR.mkdir(exist_ok=True)


In [ ]:
df = pd.read_csv(DATA_PATH)
df.head()


In [ ]:
df.shape, df.describe(include="all").T.head(20)


In [ ]:
print("class balance:"); 
print(df["diabetes"].value_counts(normalize=True).round(3))
print("\nmissing fraction (top 10):")
print(df.isna().mean().sort_values(ascending=False).head(10))


In [ ]:
num_cols = [c for c in df.select_dtypes(include=[np.number]).columns if c != "diabetes"]
if num_cols:
    fig, ax = plt.subplots(figsize=(6,4))
    df[num_cols[0]].hist(bins=30, ax=ax); ax.set_title(f"Histogram: {num_cols[0]}"); plt.show()
if len(num_cols) > 1:
    fig, ax = plt.subplots(figsize=(6,4))
    df[num_cols[1]].hist(bins=30, ax=ax); ax.set_title(f"Histogram: {num_cols[1]}"); plt.show()


In [ ]:
if len(num_cols) >= 2:
    corr = df[num_cols].corr(numeric_only=True)
    fig, ax = plt.subplots(figsize=(6,5))
    im = ax.imshow(corr.values, aspect="auto")
    ax.set_xticks(range(len(num_cols))); ax.set_xticklabels(num_cols, rotation=90)
    ax.set_yticks(range(len(num_cols))); ax.set_yticklabels(num_cols)
    ax.set_title("Correlation (numeric)"); plt.tight_layout(); plt.show()


In [ ]:
def normalize_smoking(d: pd.DataFrame) -> pd.DataFrame:
    d = d.copy()
    col = "smoking_history"
    if col in d.columns:
        v = d[col].astype(str).str.strip().str.lower()
        v = v.replace({"ever":"former","not current":"former","no info":"unknown","none":"unknown","nan":"unknown"})
        v = v.where(v.isin(["never","former","current","unknown"]), "unknown")
        d[col] = v
        d["ever_smoked"] = v.isin(["former","current"]).astype(int)
    else:
        d["smoking_history"] = "unknown"
        d["ever_smoked"] = 0
    return d

df = normalize_smoking(df)
y = df["diabetes"].astype(int)
X = df.drop(columns=["diabetes"])

numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
binary_features = [c for c in ["hypertension","heart_disease","ever_smoked"] if c in X.columns]
cat_features = [c for c in X.select_dtypes(include=["object","category"]).columns if c not in binary_features]

numeric_tf = Pipeline([("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())])
binary_tf = "passthrough"
cat_tf = Pipeline([("imputer", SimpleImputer(strategy="most_frequent")), ("ohe", OneHotEncoder(handle_unknown="ignore"))])

preprocessor = ColumnTransformer([("num", numeric_tf, numeric_features),
                                  ("bin", binary_tf, binary_features),
                                  ("cat", cat_tf, cat_features)], remainder="drop")


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)

models = {
    "logistic": LogisticRegression(max_iter=2000, solver="lbfgs"),
    "rf": RandomForestClassifier(n_estimators=400, random_state=42, n_jobs=-1),
    "xgb": XGBClassifier(n_estimators=300, learning_rate=0.05, max_depth=4,
                         subsample=0.9, colsample_bytree=0.9, reg_lambda=1.0,
                         reg_alpha=0.0, n_jobs=-1, objective="binary:logistic",
                         eval_metric="logloss", random_state=42),
}

rows = []
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for name, clf in models.items():
    pipe_m = Pipeline([("preprocessor", preprocessor), ("classifier", clf)])
    aucs = cross_val_score(pipe_m, X_train, y_train, cv=cv, scoring="roc_auc")
    rows.append([name, aucs.mean(), aucs.std()])

cmp = pd.DataFrame(rows, columns=["model","cv_auc_mean","cv_auc_std"]).sort_values("cv_auc_mean", ascending=False).reset_index(drop=True)
print(cmp)
best_name = cmp.loc[0,"model"]; print("keep:", best_name); print("drop:", cmp.loc[1:,"model"].tolist())

clf_best = models[best_name]
pipe = Pipeline([("preprocessor", preprocessor), ("classifier", clf_best)])
pipe.fit(X_train, y_train)
proba_test = pipe.predict_proba(X_test)[:,1]
print("test AUC (uncalibrated):", round(roc_auc_score(y_test, proba_test), 3))


In [ ]:
if best_name == "xgb":
    grid = [
        dict(max_depth=3, n_estimators=250, learning_rate=0.07, subsample=0.9, colsample_bytree=0.9),
        dict(max_depth=4, n_estimators=300, learning_rate=0.05, subsample=0.9, colsample_bytree=0.9),
        dict(max_depth=5, n_estimators=350, learning_rate=0.04, subsample=0.9, colsample_bytree=0.8),
    ]
    best_auc = -1.0; best_params = None
    for p in grid:
        xgb_t = XGBClassifier(**p, reg_lambda=1.0, reg_alpha=0.0, n_jobs=-1,
                              objective="binary:logistic", eval_metric="logloss", random_state=42)
        pipe_t = Pipeline([("preprocessor", preprocessor), ("classifier", xgb_t)])
        aucs = cross_val_score(pipe_t, X_train, y_train, cv=cv, scoring="roc_auc")
        m = aucs.mean(); print("try", p, "cv_auc:", round(m,3))
        if m > best_auc: best_auc, best_params = m, p
    if best_params is not None:
        print("tuned:", best_params, "| cv_auc:", round(best_auc,3))
        clf_best = XGBClassifier(**best_params, reg_lambda=1.0, reg_alpha=0.0, n_jobs=-1,
                                 objective="binary:logistic", eval_metric="logloss", random_state=42)
        pipe = Pipeline([("preprocessor", preprocessor), ("classifier", clf_best)])
        pipe.fit(X_train, y_train)
        proba_test = pipe.predict_proba(X_test)[:,1]
        print("test AUC (uncalibrated, tuned):", round(roc_auc_score(y_test, proba_test), 3))


In [ ]:
cal = CalibratedClassifierCV(pipe, method="isotonic", cv=5)
cal.fit(X_train, y_train)

proba_uncal = pipe.predict_proba(X_test)[:,1]
proba_cal = cal.predict_proba(X_test)[:,1]

print("brier uncal:", round(brier_score_loss(y_test, proba_uncal), 4))
print("brier cal  :", round(brier_score_loss(y_test, proba_cal), 4))

prob_true_uncal, prob_pred_uncal = calibration_curve(y_test, proba_uncal, n_bins=10, strategy="quantile")
prob_true_cal, prob_pred_cal = calibration_curve(y_test, proba_cal, n_bins=10, strategy="quantile")

fig, ax = plt.subplots(figsize=(6,5))
ax.plot([0,1],[0,1],"--", alpha=0.5)
ax.plot(prob_pred_uncal, prob_true_uncal, marker="o", label="uncal")
ax.plot(prob_pred_cal, prob_true_cal, marker="o", label="cal")
ax.set_xlabel("predicted"); ax.set_ylabel("observed"); ax.set_title("Calibration"); ax.legend()
fig.tight_layout(); fig.savefig(RESULTS_DIR / "calibration.png", dpi=150); plt.show()


In [ ]:
ths = np.linspace(0.05, 0.95, 19)
rows = []; best = (None, -1.0)
for t in ths:
    y_hat = (proba_cal >= t).astype(int)
    p, r, f1, _ = precision_recall_fscore_support(y_test, y_hat, average="binary", zero_division=0)
    rows.append([t, p, r, f1])
    if f1 > best[1]: best = (t, f1)

sweep = pd.DataFrame(rows, columns=["threshold","precision","recall","f1"])
best_t, best_f1 = best
print("best threshold:", round(best_t,3), "f1:", round(best_f1,3))
fig, ax = plt.subplots(figsize=(6,4))
ax.plot(sweep["threshold"], sweep["f1"], marker="o"); ax.set_xlabel("threshold"); ax.set_ylabel("F1"); ax.set_title("Threshold sweep")
fig.tight_layout(); fig.savefig(RESULTS_DIR / "threshold_curve.png", dpi=150); plt.show()

cm = confusion_matrix(y_test, (proba_cal >= best_t).astype(int))
cm


In [ ]:
clf = pipe.named_steps["classifier"]
pp = pipe.named_steps["preprocessor"]
Xtr = pp.transform(X_train)
try:
    explainer = shap.TreeExplainer(clf)
    sv = explainer(Xtr[:1])
    shap.plots.waterfall(sv[0], show=False)
    plt.gcf().set_size_inches(8,5); plt.tight_layout()
    plt.savefig(RESULTS_DIR / "shap_waterfall.png", dpi=150); plt.show()
except Exception as e:
    print("SHAP warning:", e)


In [ ]:
out_path = MODELS_DIR / "calibrated_diabetes_model_v5.pkl"
joblib.dump(cal, out_path); out_path
